In [2]:
from json import dumps
def safeVisNetworkJSONToHTMLFile(jsonData, htmlFilePath):
    with open(htmlFilePath, "w") as htmlFile:
        htmlFile.write(
            f"""
            <!DOCTYPE html>
            <html lang="en-US">
            
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1">
                <title>Inline vis</title>
            </head>
            
            <body>
                <div id="mynetwork" style="width:100vw; height:100vh;"></div>
                <script src=" https://cdn.jsdelivr.net/npm/vis-data@7.1.9/peer/umd/vis-data.min.js "></script>
                <script src=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/peer/umd/vis-network.min.js "></script>
                <link href=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/styles/vis-network.min.css " rel="stylesheet">
                <script>
                    var jsonData = {dumps(jsonData)};
                    // create a network
                    var container = document.getElementById("mynetwork");
                    var options = {{}};
                    var network = new vis.Network(container, jsonData, options);
                </script>
            </body>
            
            </html>
            """
        )

In [3]:

from urllib.parse import quote
import latextools
import drawsvg as draw


In [4]:
def convertFormulaToSVGURL(formula):
    latex_eq = latextools.render_snippet(
        "$" + formula + "$",
        commands=[latextools.cmd.all_math])
    svg_eq = latex_eq.as_svg()
    svgwidth = 300
    svgheight = 100
    d = draw.Drawing(svgwidth, svgheight, origin='center', displayInline=False)
    d.append(draw.Ellipse(0, 0, svgwidth/2 - 1, svgheight/2 - 1, fill='yellow', stroke='black', stroke_width=2))
    d.draw(svg_eq, x=0, y=0, center=True, scale=0.8*min(svgwidth/svg_eq.width, svgheight/svg_eq.height))
    simpleSvg = d.as_svg()
    return f"data:image/svg+xml,{quote(simpleSvg)}"

In [5]:
def convertFormulaSymbolDefinitionToSVGURL(symbol):
    latex_eq = latextools.render_snippet(
        "where $" + symbol + "$ is",
        commands=[latextools.cmd.all_math])
    svg_eq = latex_eq.as_svg()
    # Fit the symbol definition into a rectangle
    d = draw.Drawing(svg_eq.width + 6, svg_eq.height + 6, origin='center', displayInline=False)
    d.append(draw.Rectangle(-d.width/2, -d.height/2, d.width, d.height, fill='lightgreen', stroke='black', stroke_width=2))
    d.draw(svg_eq, x=0, y=0, center=True)
    simpleSvg = d.as_svg()
    return f"data:image/svg+xml,{quote(simpleSvg)}"

In [6]:
def convertFormularNetworkToVisNetworkJSON(formularNetwork):
    """
    FormularNetwork is a list of triples (subj, pred, obj).
    Subj and obj are eather formulas ("f: ... ") or text ("t: ...").
    The pred is one of the following relations:
    - "is defined by" ("!")
    - "equals" ("=")
    - "where ??? is" (": ...")
    """
    nodes = []
    edges = []
    indexPerTerm = {}
    indexCounter = 0
    for subj, pred, obj in formularNetwork:
        for term in subj, obj:
            if term not in indexPerTerm:
                indexCounter += 1
                indexPerTerm[term] = indexCounter
                if term.startswith("f: "):
                    formulaURL = convertFormulaToSVGURL(term[3:])
                    nodes.append({
                        "id": indexCounter, 
                        "shape": "image",
                        "image": formulaURL
                    })
                elif term.startswith("t: "):
                    nodes.append({
                        "id": indexCounter, 
                        "label": term[3:], 
                        "font": {"size": 20}, 
                        "color" : {"background": "rgba(140, 220, 255, 255)", 
                                    "border": "rgba(0, 0, 255, 255)"}})
                else:
                    raise ValueError(f"Unknown term type: {term}")
        indexCounter += 1
        if pred == "!":
            nodes.append({
                "id": indexCounter, 
                "label": "is defined by", 
                "font": {"size": 16},
                "color" : {"background": "rgba(255, 200, 100, 255)", 
                            "border": "darkorange"},
                "shape" : "box"})
            edges.append({"from": indexPerTerm[subj], "to": indexCounter, "color": "rgba(200, 100, 60, 255)", "width": 3})
            edges.append({"from": indexCounter, "to": indexPerTerm[obj], "arrows": "to", "color": "rgba(200, 100, 60, 255)", "width": 3})
        elif pred == "=":
            nodes.append({
                "id": indexCounter, 
                "label": "equals", 
                "font": {"size": 16},
                "color" : {"background": "rgba(255, 100, 100, 255)", 
                            "border": "darkred"},
                "shape" : "box"})
            edges.append({"from": indexCounter, "to": indexPerTerm[subj], "color": "rgba(200, 60, 60, 255)", "width": 3, "arrows": "to"})
            edges.append({"from": indexCounter, "to": indexPerTerm[obj], "color": "rgba(200, 60, 60, 255)", "width": 3, "arrows": "to"})
        elif pred.startswith(": "):
            formulaURL = convertFormulaSymbolDefinitionToSVGURL(pred[2:])
            nodes.append({
                "id": indexCounter, 
                "shape": "image",
                "size": 11,
                "image": formulaURL})
            edges.append({"from": indexPerTerm[subj], "to": indexCounter, "color": "rgba(100, 200, 60, 255)", "width": 3})
            edges.append({"from": indexCounter, "to": indexPerTerm[obj], "arrows": "to", "color": "rgba(100, 200, 60, 255)", "width": 3})
    return {"nodes": nodes, "edges": edges}

In [8]:
h_w = r'f: \hbar \omega'
hv = r'f: h f'
E = r'f: E'
energy = "t: Energy of the Photon"
formulaNetwork = [
    #[energy, "!", h_w],
    [h_w, "=", hv],
    #[energy, "!", hv],
    [hv, "=", E],
    [h_w, "=", E],
    [E, ": E", energy],
    [hv, ": h", "t: Planck's constant"],
    [hv, ": f", "t: Frequency of the photon"],
    [h_w, ": \hbar", "t: Reduced Planck's constant"],
    [h_w, ": \omega", "t: Angular frequency of the photon"]
    ]
safeVisNetworkJSONToHTMLFile(convertFormularNetworkToVisNetworkJSON(formulaNetwork), "output.html")

In [8]:
from openai import OpenAI
from random import randint
from os import environ
from pathlib import Path
environ["OPENAI_API_KEY"] = Path("~/.openaiapikey").expanduser().read_text().strip()

openaiClient = OpenAI()
def gpt_3_5_turbo_completion(query):
    answer = openaiClient.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def gpt_4_turbo_completion(query):
    answer = openaiClient.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def tryRecieveAnswer(query, completionFunction = gpt_4_turbo_completion, answerConversion = lambda x: True, maxTries = 10):
    tryNumber = 0
    while tryNumber < maxTries:
        answer = completionFunction(query)
        try:
            answer = answerConversion(answer)
            return (answer, True)
        except:
            pass
        tryNumber += 1
    print(f"Failed to recieve answer for query: {query}")
    return (None, False)

In [54]:
gpt_4_turbo_completion("Give me the latex code for the formula of the energy of a photon. Return nothing but the formula surrounded by $ signs.")

'$E = h \\nu$'

In [9]:
from json import loads
def findFormulaFor(concept):
    query = """Example of a formula for the Energy of a Photon: ["$E = \\hbar \\omega$ =  h f", {"$E$": "Energy of the Photon", "$\\hbar$": "Reduced Planck\'s constant", "$\\omega$": "Angular frequency of the photon", "$h$": "Planck\'s constant", "$f$": "Frequency of the photon"}]
            give me a formula for \"""" + concept +"""\" with the explanations of all symbols formatted in the same way. The explanations of the symbols should be concepts written in a compact form. Return nothing but the result [formula, symbol explanations]."""
    def answerConversion(answer):
        answer = answer.replace("\\", "\\\\") # Escape backslashes
        answer = loads(answer)
        assert type(answer) == list and len(answer) == 2
        assert type(answer[0]) == str
        assert type(answer[1]) == dict
        return answer
    return tryRecieveAnswer(query, answerConversion = answerConversion)

In [47]:
def isPartOfFormular(symbol, formula):
    if not symbol.isalpha():
        return symbol in formula
    formulaCompounts = []
    i = 0
    linkNextParts = False
    while i < len(formula):
        linkParts = False
        if formula[i] == "\\":
            i += 1
            compount = "\\"
            while i < len(formula) and formula[i].isalpha():
                compount += formula[i]
                i += 1
            formulaCompounts.append(compount)
        elif formula[i].isalpha():
            formulaCompounts.append(formula[i])
            i += 1
        elif formula[i].isnumeric():
            compount = formula[i]
            i += 1
            while i < len(formula) and formula[i].isnumeric():
                compount += formula[i]
                i += 1
            formulaCompounts.append(compount)
        elif formula[i] == "_":
            linkParts = True
            i += 1
        else:
            i += 1
        if linkNextParts:
            if len(formulaCompounts) > 1:
                formulaCompounts[-2] += "_" + formulaCompounts[-1]
                formulaCompounts.pop()
        linkNextParts = linkParts
        
    return symbol in formulaCompounts

In [11]:
def createFormulaNetwork(formulaAndSymbolExplanations):
    formula, symbolExplanations = formulaAndSymbolExplanations
    formulaNetwork = []
    assert formula[0] == "$" and formula[-1] == "$"
    formula = formula[1:-1]
    formulaParts = formula.split("=")
    for i in range(len(formulaParts)):
        formulaParts[i] = formulaParts[i].strip()
    for i in range(len(formulaParts) - 1):
        formulaNetwork.append([f"f: {formulaParts[i]}", "=", f"f: {formulaParts[i+1]}"])
    for formulaPart in formulaParts:
        for symbol, explanation in symbolExplanations.items():
            assert symbol[0] == "$" and symbol[-1] == "$"
            symbol = symbol[1:-1].strip()
            if isPartOfFormular(symbol, formulaPart):
                formulaNetwork.append([f"f: {formulaPart}", ": " + symbol, f"t: {explanation}"])
    return formulaNetwork

In [62]:
[formula, symbolExplanations], success = findFormulaFor("Spin of a particle")
print(formula)
print(symbolExplanations)
formulaNetwork = createFormulaNetwork([formula, symbolExplanations])
safeVisNetworkJSONToHTMLFile(convertFormularNetworkToVisNetworkJSON(formulaNetwork), "output.html")

$S = \sqrt{s(s+1)} \hbar$
{'$S$': 'Spin magnitude of the particle', '$s$': 'Spin quantum number', '$\\hbar$': "Reduced Planck's constant"}
